In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.metrics as sm
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_excel('coefficient2.xlsx')
df

,Unnamed: 0,blood,skin,Fat,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,blood.1,skin.1,Fat.1,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,Frequency(THz),refractive index,NaN,NaN,NaN,NaN,NaN,Frequency(THz),absorption coefficient(mm^-1),NaN,NaN,NaN,Path loss (blood) distance d=1mm,NaN,Path loss in dB
1,0.2,2.7108,2.487803,2.0305,NaN,NaN,NaN,0.2,11.7758,7.240005,4.0489,NaN,6.70806e+07,NaN,156.532
2,0.4,2.2105,2.246139,1.9319,NaN,NaN,NaN,0.4,16.0218,9.490481,5.8964,NaN,1.24583e+10,NaN,201.909
3,0.6,2.0427,2.165657,1.8659,NaN,NaN,NaN,0.6,18.6761,11.736955,8.0317,NaN,3.40264e+11,NaN,230.636
4,0.8,1.9573,2.111387,1.8101,NaN,NaN,NaN,0.8,20.9721,14.240531,10.0315,NaN,5.51746e+12,NaN,254.835
5,1,1.9013,2.065012,1.7649,NaN,NaN,NaN,1,23.1726,16.846053,11.7077,NaN,7.34532e+13,NaN,277.32
6,1.2,1.8588,2.023375,1.7295,NaN,NaN,NaN,1.2,25.3088,19.394905,13.0432,NaN,8.56007e+14,NaN,298.65
7,1.4,1.8239,1.986050,1.7021,NaN,NaN,NaN,1.4,27.3544,21.781510,14.0876,NaN,8675615827705397,NaN,318.766
8,1.6,1.794,1.952969,1.6809,NaN,NaN,NaN,1.6,29.2782,23.951465,14.8994,NaN,75062454482432384,NaN,337.508
9,1.8,1.7681,1.923947,1.6644,NaN,NaN,NaN,1.8,31.0583,25.887087,15.5355,NaN,547248001711980800,NaN,354.764


# Linear Regression Model with multiple variable

##### Initializing the data

In [3]:
freq = df.loc[1:, 'Unnamed: 0'] #frequency
pa_lo = df.loc[1:, 'Unnamed: 14'] #path loss (db) for 1mm distance
x = df.loc[1:, ('Unnamed: 0', 'blood', 'blood.1')] #multiple features (frequency, refractive index, absorption coefficient)

##### Fitting the model

In [4]:
model = LinearRegression().fit(x, pa_lo)
y_pred = model.predict(x)

##### Measure the accuracy

In [ ]:
r_sq = model.score(x, pa_lo)
print("Regression accuracy = {:0.2f}%".format(100 * r_sq))

##### Plotting the data

In [ ]:
plt.plot(freq, pa_lo, 'g^--', label='Output' )
plt.plot(freq, y_pred, 'r-', label= "Prediction")
plt.text(.15, 320, ("Regression accuracy = {:0.2f}%".format(100 * r_sq)))
plt.text(.15, 300, "distance = 1 mm")
plt.xlabel('Frequency (THz)')
plt.ylabel('Path Loss (dB)')
plt.title('Linear Regression model (blood cell)')
plt.legend()
plt.savefig('prediction_with_multi_var.png')

# Cross checking the accuracy by another way

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, pa_lo, random_state=1)
lm2 = LinearRegression()
lm2.fit(X_train, y_train)
y_pred = lm2.predict(X_test)
print("Regression accuracy = {:0.2f}%".format(100 * sm.r2_score(y_test, y_pred)))